<a href="https://colab.research.google.com/github/yehocoh/technical-analysis-machine-learning/blob/main/vbt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def convert_pred_to_strategy(pred_df):
    # Initialize variables
    actions = pd.Series(index=pred_df.index, dtype='int')
    holding = 0

    # Loop through predicted values
    for i, val in pred_df.iterrows():
        if holding == 0:
            # Buy if not holding
            if val[0] == 1:
                actions[i] = 1
                holding = 1
            else:
                actions[i] = 0
        else:
            # Sell if predicted to sell
            if val[0] == -1:
                actions[i] = -1
                holding = 0
            else:
                actions[i] = 0

    return actions

In [ ]:
# %pip install vectorbt
# %pip install --upgrade urllib3
# %pip install yfinance
# %pip install pip install TA-Lib

In [ ]:
import vectorbt as vbt
from datetime import datetime, timedelta
import talib
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

days_= 250
symbols = [
           'QQQ'
           ,"SPY"
           ,"AAPL"
           , "MSFT"
           , "GOOG"
           , "AMZN"
           , "META"
           , "TSLA"
           , "NVDA"
           , "PYPL"
          ]

start_ = (datetime.now()- timedelta(days=days_))
end_ = datetime.now()

symbol_data = []

for symbol in symbols:
    print(symbol)
    data = yf.download(
                        symbol
                        , period="max"
                        , interval="1D"
                        , start=start_
                        , end=end_
                        , progress=False
                                            )

    # Loading the pattern rules in technical analysis by the TA-LIB library
    results = []
    cols = []
    for attr in dir(talib):
      if attr[:3] == 'CDL' :
        res = getattr(talib,attr)(data.Open,data.High,data.Low,data.Close)
        results.append(res)
        cols.append(attr)

    # The arrangement of the data in DF
    patterns = pd.DataFrame(results).transpose()
    patterns.columns = cols
    # print(patterns)

    # pct_change
    returns = data.Close.pct_change()

    # Running a logistic regression model
    sft = -1
    oos = 10

  # MODEL FIT
    model = LogisticRegression(max_iter=5000,tol=0.01)
    model.fit(patterns.iloc[sft+1:-oos],np.sign(returns.shift(sft).iloc[sft+1:-oos]))

    # Model prediction
    pred = model.predict(patterns.iloc[-oos:])
    real = returns.shift(sft).iloc[-oos:]

    df = pd.DataFrame(data.iloc[-oos:])
    df['symbol'] = symbol
    df['pct_change'] = returns.iloc[-oos:]
    df['shift'] = returns.shift(sft).iloc[-oos:]
    df['pred'] = pred
    df['strategy'] = convert_pred_to_strategy(df[['pred']])
    df = pd.merge(df, patterns.iloc[-oos:], left_index=True, right_index=True)


    symbol_data.append(df)

result_patterns = pd.concat(symbol_data)
result_patterns.sort_values('Date')


# finalCandleDf = result_patterns[["Close","symbol","pred","strategy"]].sort_values('Date').rename(columns={'strategy': 'signals','Close' : 'Price'})
finalCandleDf = result_patterns[["pred","symbol"]]
# finalCandleDf

show =result_patterns[result_patterns.index == datetime.now()- timedelta(days=5) ]
result_patterns


QQQ
SPY
AAPL
MSFT
GOOG
AMZN
META
TSLA
NVDA
PYPL


,Open,High,Low,Close,Adj Close,Volume,symbol,pct_change,shift,pred,...,CDLSPINNINGTOP,CDLSTALLEDPATTERN,CDLSTICKSANDWICH,CDLTAKURI,CDLTASUKIGAP,CDLTHRUSTING,CDLTRISTAR,CDLUNIQUE3RIVER,CDLUPSIDEGAP2CROWS,CDLXSIDEGAP3METHODS
Date,,,,,,,,,,,,,,,,,,,,,
2023-06-20,365.950012,368.320007,363.890015,366.899994,366.899994,49204700,QQQ,-0.002799,-0.013628,-1.0,...,100,0,0,0,0,0,0,0,0,0
2023-06-21,365.739990,366.179993,360.959991,361.899994,361.899994,53378300,QQQ,-0.013628,0.011799,1.0,...,0,0,0,0,0,0,0,0,0,0
2023-06-22,360.630005,366.329987,360.220001,366.170013,366.170013,47603000,QQQ,0.011799,-0.009913,1.0,...,0,0,0,0,0,0,0,0,0,0
2023-06-23,362.209991,364.880005,360.820007,362.540009,362.540009,48873900,QQQ,-0.009913,-0.013405,-1.0,...,100,0,0,0,0,0,0,0,0,0
2023-06-26,362.000000,364.839996,357.589996,357.679993,357.679993,52685500,QQQ,-0.013405,0.017194,-1.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-27,66.870003,67.099998,65.779999,66.230003,66.230003,16371200,PYPL,-0.003161,-0.003020,-1.0,...,0,0,0,0,0,0,0,0,0,0
2023-06-28,66.050003,66.480003,65.620003,66.029999,66.029999,15390600,PYPL,-0.003020,-0.002575,-1.0,...,-100,0,0,0,0,0,0,0,0,0
2023-06-29,66.300003,67.339996,65.620003,65.860001,65.860001,14894200,PYPL,-0.002575,0.013210,1.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# List to store the data for each ETF
comb_price = pd.DataFrame()

# For loop to retrieve data for each ETF symbol
for symbol in symbols:
    # print(symbol)
    comb_price[symbol] = yf.download(
                                    symbol,
                                    start=start_,
                                    end=end_,
                                    progress=False
                                                    ).get('Close').iloc[-oos:]
# print(comb_price)

finalCandleDf_pivot_buy = pd.pivot_table(finalCandleDf, values='pred', index='Date', columns='symbol')
finalCandleDf_pivot_buy = finalCandleDf_pivot_buy.applymap(lambda x: True if x == 1 else False)
finalCandleDf_pivot_buy= finalCandleDf_pivot_buy.loc[:, symbols]

finalCandleDf_pivot_sell = pd.pivot_table(finalCandleDf, values='pred', index='Date', columns='symbol')
finalCandleDf_pivot_sell =finalCandleDf_pivot_sell.applymap(lambda x: True if x == -1 else False)
finalCandleDf_pivot_sell= finalCandleDf_pivot_sell.loc[:, symbols]

entries = finalCandleDf_pivot_buy
# print(entries)

exits = finalCandleDf_pivot_sell
# print(exits)

pf = vbt.Portfolio.from_signals(comb_price,
                                entries,
                                exits,
                                freq="D",
                                init_cash=10_000,
                                fees=0.0001,
                                )

print(pf.returns_stats())

Start                        2023-06-20 00:00:00
End                          2023-07-03 00:00:00
Period                          10 days 00:00:00
Total Return [%]                        1.702976
Benchmark Return [%]                    0.553845
Annualized Return [%]                1917.354302
Annualized Volatility [%]              24.217883
Max Drawdown [%]                        2.329061
Max Drawdown Duration            5 days 02:24:00
Sharpe Ratio                            1.987329
Calmar Ratio                       190305.231568
Omega Ratio                            74.932424
Sortino Ratio                         631.121227
Skew                                    0.626714
Kurtosis                                 1.72372
Tail Ratio                             56.290891
Common Sense Ratio                  10527.322183
Value at Risk                          -0.012738
Alpha                                   13.63703
Beta                                    0.644035
Name: agg_func_mean,

/Users/yehocoh/Library/Python/3.9/lib/python/site-packages/vectorbt/portfolio/base.py:4789: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x10562ce50>. Pass column to select a single column/group.



In [ ]:
pf.orders.records_readable

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,"(QQQ, QQQ)",2023-06-21,27.629180,361.899994,0.999900,Buy
1,1,"(QQQ, QQQ)",2023-06-23,27.629180,362.540009,1.001668,Sell
2,2,"(QQQ, QQQ)",2023-06-29,27.527226,363.809998,1.001468,Buy
3,3,"(QQQ, QQQ)",2023-06-30,27.527226,369.420013,1.016911,Sell
4,4,"(QQQ, QQQ)",2023-07-03,27.457059,370.290009,1.016707,Buy
5,5,"(SPY, SPY)",2023-06-20,22.871587,437.179993,0.999900,Buy
6,6,"(SPY, SPY)",2023-06-21,22.871587,434.940002,0.994777,Sell
7,7,"(SPY, SPY)",2023-06-22,22.784766,436.510010,0.994578,Buy
8,8,"(SPY, SPY)",2023-06-28,22.784766,436.390015,0.994304,Sell
9,9,"(SPY, SPY)",2023-06-29,22.690777,438.109985,0.994106,Buy


In [ ]:
pf.total_return().vbt.plot()

FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': 'scatter',
              'uid': '44204b25-3267-47a0-8273-2c15613d5d9d',
              'x': [('QQQ', 'QQQ'), ('SPY', 'SPY'), ('AAPL', 'AAPL'), ('MSFT',
                    'MSFT'), ('GOOG', 'GOOG'), ('AMZN', 'AMZN'), ('META', 'META'),
                    ('TSLA', 'TSLA'), ('NVDA', 'NVDA'), ('PYPL', 'PYPL')],
              'y': array([ 0.01670746,  0.00573619, -0.00510274,  0.00853759,  0.0100701 ,
                           0.00307071,  0.00728148,  0.15481127, -0.03178075,  0.00096634])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
        

In [ ]:
print(pf.stats())

Start                               2023-06-20 00:00:00
End                                 2023-07-03 00:00:00
Period                                 10 days 00:00:00
Start Value                                     10000.0
End Value                                  10170.297649
Total Return [%]                               1.702976
Benchmark Return [%]                           0.553845
Max Gross Exposure [%]                            100.0
Total Fees Paid                                4.790777
Max Drawdown [%]                               2.329061
Max Drawdown Duration                   5 days 02:24:00
Total Trades                                        2.7
Total Closed Trades                                 2.1
Total Open Trades                                   0.6
Open Trade PnL                                99.511734
Win Rate [%]                                  56.666667
Best Trade [%]                                 1.660383
Worst Trade [%]                               -0

/var/folders/nv/l4t7vfpj1dd6k81r6lwx7ym80000gn/T/ipykernel_25235/1918761186.py:1: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x10562ce50>. Pass column to select a single column/group.

